# Convolutional Neural Network
- MNIST data
- 3 convolutional layers
- 2 fully connected layers

## 1. Settings
### 1) Import required libraries

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
from visdom import Visdom
viz = Visdom()

### 2) Set hyperparameters

In [2]:
batch_size = 16
learning_rate = 0.0002
num_epoch = 10

## 2. Data

### 1) Download Data

In [3]:
mnist_train = dset.MNIST("./", train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
mnist_test = dset.MNIST("./", train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

### 2) Check Dataset

In [4]:
print(mnist_train.__getitem__(0)[0].size(), mnist_train.__len__())
mnist_test.__getitem__(0)[0].size(), mnist_test.__len__()

torch.Size([1, 28, 28]) 60000


(torch.Size([1, 28, 28]), 10000)

### 3) Set DataLoader

In [5]:
train_loader = torch.utils.data.DataLoader(mnist_train,batch_size=batch_size, shuffle=True,num_workers=2,drop_last=True)
test_loader = torch.utils.data.DataLoader(mnist_test,batch_size=batch_size, shuffle=False,num_workers=2,drop_last=True)

## 3. Model & Optimizer

### 1) CNN Model

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(1,16,5),
            nn.ReLU(),
            nn.Conv2d(16,32,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32,64,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(64*3*3,100),
            nn.ReLU(),
            nn.Linear(100,10)
        )       
        
    def forward(self,x):
        out = self.layer(x)
        out = out.view(batch_size,-1)
        out = self.fc_layer(out)

        return out

model = CNN().cuda()

### 2) Loss func & Optimizer

In [7]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## 4. Train 

In [8]:
for i in range(num_epoch):
    for j,[image,label] in enumerate(train_loader):
        x = Variable(image).cuda()
        y_= Variable(label).cuda()
        
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output,y_)
        loss.backward()
        optimizer.step()
        
        if j % 1000 == 0:
            print(loss)          

Variable containing:
 2.2991
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 2.3071
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 2.2968
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 2.2922
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 2.2815
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 2.3111
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 2.2976
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 2.2919
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 2.3026
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 2.2916
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 2.2892
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 2.2883
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 2.2887
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 2.2822
[torch.cuda.FloatTenso

Process Process-13:
Process Process-14:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/gunho/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/gunho/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/gunho/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/gunho/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 34, in _worker_loop
    r = index_queue.get()
  File "/home/gunho/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/gunho/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/queues.py", line 341, in get
    with self._rlock:
KeyboardInterrupt
  File "/home/gunho/anaconda3/envs/pytorch/lib/

KeyboardInterrupt: 

In [9]:
#param_list = list(model.parameters())
#print(param_list)

## 5. Test

In [10]:
correct = 0
total = 0

for image,label in test_loader:
    x = Variable(image,volatile=True).cuda()
    y_= Variable(label).cuda()

    output = model.forward(x)
    _,output_index = torch.max(output,1)
        
    total += label.size(0)
    correct += (output_index == y_).sum().float()
    
print("Accuracy of Test Data: {}".format(100*correct/total))

Accuracy of Test Data: Variable containing:
 67.8300
[torch.cuda.FloatTensor of size 1 (GPU 0)]

